In [307]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
import pandas as pd
import numpy as np
!pip install cpi
import cpi

# **Primary data processing**
Let's open our dataset and make some changes to it for further analysis

In [308]:
dfm = pd.read_csv('./tmdb_5000_movies.csv')
dfm['release_date'] = pd.to_datetime(dfm['release_date'])

num_cols = ['budget', 'popularity', 'runtime', 'vote_average', 'vote_count']
id_cols = ['id']

cat_cols = ['original_language']

target = ['revenue']

dfm.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [309]:
dfm = dfm.drop(dfm[(dfm.runtime == 0) | (dfm.revenue == 0) | (dfm.budget == 0)].index, axis=0)

print(len(dfm))
dfm.describe()

3229


,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,3.229000e+03,3229.000000,3229.000000,3.229000e+03,3229.000000,3229.000000,3229.000000
mean,4.065444e+07,44780.705791,29.033689,1.212430e+08,110.724373,6.309353,977.287395
std,4.439674e+07,74609.434723,36.165730,1.863029e+08,20.965694,0.873891,1414.309577
min,1.000000e+00,5.000000,0.019984,5.000000e+00,41.000000,0.000000,0.000000
25%,1.050000e+07,4958.000000,10.446722,1.700000e+07,96.000000,5.800000,178.000000
50%,2.500000e+07,11451.000000,20.410354,5.518472e+07,107.000000,6.300000,471.000000
75%,5.500000e+07,45272.000000,37.335721,1.462920e+08,121.000000,6.900000,1148.000000
max,3.800000e+08,417859.000000,875.581305,2.787965e+09,338.000000,8.500000,13752.000000


In [310]:
#Data columns
dfm['year'] = pd.DatetimeIndex(dfm['release_date']).year
dfm['month'] = pd.DatetimeIndex(dfm['release_date']).month
#Inflation
"""dfm['revenue'] = dfm.apply(lambda x: cpi.inflate(x.revenue, x.year), axis=1)
dfm['budget'] = dfm.apply(lambda x: cpi.inflate(x.budget, x.year), axis=1)"""

"dfm['revenue'] = dfm.apply(lambda x: cpi.inflate(x.revenue, x.year), axis=1)\ndfm['budget'] = dfm.apply(lambda x: cpi.inflate(x.budget, x.year), axis=1)"

In [311]:
#Change type of original language
dfm.original_language[dfm.original_language != 'en'] = 0
dfm.original_language[dfm.original_language != 0] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
import json

#Translate data to acceptable format by converting a JSON columns
prime_genre = pd.json_normalize(dfm.genres.apply(json.loads).apply(lambda x: x[0] if (x != []) else {'id': 0, 'name': 'None'}))
secondary_genre = pd.json_normalize(dfm.genres.apply(json.loads).apply(lambda x: x[1] if (len(x) > 1) else {'id': 0, 'name': 'None'}))

#genres
dfm['prime_genre_name'] = prime_genre['name']
dfm['secondary_genre_name'] = secondary_genre['name']

#production country
dfm['production_country_id'] = pd.json_normalize(dfm.production_countries.apply(json.loads).apply(lambda x: x[0] if (x != []) else {'iso_3166_1': 'None', 'name': 'Unknown'}))['iso_3166_1']

#production companies
dfm['production_company_name'] = pd.json_normalize(dfm.production_companies.apply(json.loads).apply(lambda x: x[0] if (x != []) else {'id': 'None', 'name': 'NaN'}))['name']
#dfm['second_production_company'] = pd.json_normalize(dfm.production_companies.apply(json.loads).apply(lambda x: x[1] if (len(x) > 1) else {'id': 'None', 'name': 'NaN'}))['name']

id_cols += ['production_country_id']
dfm = dfm.drop(['genres', 'homepage', 'original_title',
                'production_countries', 'production_companies', 'keywords',
                "tagline",	"title", 'overview',
                'spoken_languages'], axis=1)

In [313]:
num_cols.remove('popularity')
num_cols.remove('vote_count')
dfm[num_cols].sample(10)

,budget,runtime,vote_average
3035,10000000,84.0,5.6
2675,14000000,125.0,6.6
236,115000000,134.0,5.8
2017,22000000,127.0,6.9
2665,14200000,139.0,6.9
825,55000000,98.0,6.1
1574,30000000,99.0,5.8
3958,3000000,83.0,4.9
4437,900000,96.0,6.1
1375,35000000,100.0,5.8


In [0]:
a = pd.DataFrame.from_dict({'genres': dfm['prime_genre_name']})
b = pd.DataFrame.from_dict({'genres': dfm['secondary_genre_name']})

In [0]:
genres = pd.get_dummies(a) + pd.get_dummies(b)
dfm = pd.concat([dfm, genres], axis=1)

In [316]:
dfm = dfm.drop(['prime_genre_name', 'secondary_genre_name'], axis=1)
dfm.head(3)

,budget,id,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,year,month,production_country_id,production_company_name,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_None,genres_Romance,genres_Science Fiction,genres_Thriller,genres_War,genres_Western
0,237000000,19995,1,150.437577,2009-12-10,2787965087,162.0,Released,7.2,11800,2009,12,US,Ingenious Film Partners,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,300000000,285,1,139.082615,2007-05-19,961000000,169.0,Released,6.9,4500,2007,5,US,Walt Disney Pictures,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,245000000,206647,1,107.376788,2015-10-26,880674609,148.0,Released,6.3,4466,2015,10,GB,Columbia Pictures,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [317]:
#Now change a company format
dfm['company_portfolio'] = dfm.groupby('production_company_name')['production_company_name'].transform('count')
dfm.production_company_name[dfm.company_portfolio < 2] = 'Noname company'
#dfm.second_production_company[dfm.groupby('second_production_company')['second_production_company'].transform('count') < 2] = 'Noname company'
dfm.sample(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,budget,id,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,year,month,production_country_id,production_company_name,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_None,genres_Romance,genres_Science Fiction,genres_Thriller,genres_War,genres_Western,company_portfolio
692,150000000,9982,1,47.973995,2005-11-04,314432665,81.0,Released,5.6,944,2005,11,GB,Regency Enterprises,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,23.0
458,80000000,41515,1,11.187696,2010-12-11,201584141,80.0,Released,5.2,220,2010,12,CZ,The Weinstein Company,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24.0
3004,10000000,1832,1,14.543435,1999-11-12,30652890,130.0,Released,6.8,823,1999,11,GB,United Artists,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,41.0
2345,18000000,25643,1,13.472365,2009-09-18,36133014,109.0,Released,5.6,180,2009,9,US,Fox Searchlight Pictures,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,31.0
4080,2000000,92182,1,11.666320,2011-01-21,13101672,85.0,Released,5.2,125,2011,1,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
226,120000000,42888,1,11.137655,2010-12-17,48668907,121.0,Released,4.9,223,2010,12,US,Universal Pictures,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,185.0
1325,40000000,14325,1,3.878230,2008-10-10,9793496,130.0,Released,7.1,71,2008,10,CH,Epsilon Motion Pictures,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,8.0
1568,30000000,59967,1,46.377628,2012-09-26,47042000,118.0,Released,6.6,4697,2012,9,DE,The Weinstein Company,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,24.0
107,160000000,68735,1,63.148529,2016-05-25,433677183,123.0,Released,6.3,2268,2016,5,US,Paramount Pictures,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,200.0
1116,42000000,37233,1,23.736535,1993-06-30,270248367,154.0,Released,6.6,501,1993,6,US,Icon Productions,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0


In [0]:
c = pd.DataFrame.from_dict({'company': dfm['production_company_name']})
#c1 = pd.DataFrame.from_dict({'company': dfm['second_production_company']})

company = pd.get_dummies(c) #+ pd.get_dummies(c1)
dfm = pd.concat([dfm, company], axis=1)

In [319]:
num_cols = ['budget', 'vote_average', 'runtime', 'company_portfolio', 'year', 'month']
cat_cols = [i for i in list(map(lambda x: x if x.startswith('company') or x.startswith('genres') else None, dfm.columns)) if i]

dfm['profitable'] = (dfm.revenue / dfm.budget) > 2.5
dfm['foreign'] = (dfm.production_country_id) != 'US'

cat_cols2 = ['original_language', 'profitable', 'foreign']

#LOGARITHM IT!
"""dfm.budget = np.log(dfm.budget)
dfm.revenue = np.log(dfm.revenue)"""

'dfm.budget = np.log(dfm.budget)\ndfm.revenue = np.log(dfm.revenue)'

# SKlearn first tests

In [0]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, make_scorer, accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
import sklearn.preprocessing

In [321]:
dfm.company_portfolio = dfm.company_portfolio.fillna(1)
dfm[cat_cols + cat_cols2 + num_cols]

,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_None,genres_Romance,genres_Science Fiction,genres_Thriller,genres_War,genres_Western,company_portfolio,company_1492 Pictures,company_2929 Productions,company_40 Acres & A Mule Filmworks,company_Aardman Animations,company_Alcon Entertainment,company_Alliance Films,company_Amblin Entertainment,company_American Zoetrope,company_Anonymous Content,company_Artisan Entertainment,company_Atlas Entertainment,company_BBC Films,company_BIM Distribuzione,company_Bac Films,company_Bazelevs Production,company_Bazmark Films,company_Beacon Communications,company_Bedford Falls Productions,company_Beijing New Picture Film Co. Ltd.,company_Bel Air Entertainment,...,company_This Is That Productions,company_Tig Productions,company_Tim Burton Productions,company_Touchstone Pictures,company_Tree Line Films,company_TriStar Pictures,company_Twentieth Century Fox Film Corporation,company_UK Film Council,company_USA Films,company_United Artists,company_Universal Pictures,company_Universal Studios,company_Vertigo Entertainment,company_Village Roadshow Pictures,company_Virtual Studios,company_Walt Disney,company_Walt Disney Animation Studios,company_Walt Disney Pictures,company_Walt Disney Productions,company_Warner Bros.,company_Warner Bros. Family Entertainment,company_Warner Bros. Pictures,company_Weed Road Pictures,company_Wild Bunch,company_Wildwood Enterprises,company_WingNut Films,company_Wonderland Sound and Vision,company_Working Title Films,company_Worldview Entertainment,company_Yari Film Group,company_Zentropa Entertainments,original_language,profitable,foreign,budget,vote_average,runtime,company_portfolio,year,month
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,237000000,7.2,162.0,16.0,2009,12
1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,82.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,300000000,6.9,169.0,82.0,2007,5
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,245000000,6.3,148.0,144.0,2015,10
3,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,250000000,7.6,165.0,14.0,2012,7
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,260000000,6.1,132.0,82.0,2012,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,27000,7.4,92.0,1.0,1994,9
4788,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,12000,6.2,93.0,1.0,1972,3
4792,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True,20000,7.4,111.0,1.0,1997,11
4796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,7000,6.9,77.0,1.0,2004,10


In [322]:
"""scaler = sklearn.preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(dfm[num_cols])
scaled_df = pd.DataFrame(scaled_df, columns=dfm[num_cols].columns)
pd.concat([scaled_df.fillna(0), 
           dfm[cat_cols + cat_cols2]], 
          axis=1)"""
dfm[num_cols + cat_cols + cat_cols2]

,budget,vote_average,runtime,company_portfolio,year,month,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_None,genres_Romance,genres_Science Fiction,genres_Thriller,genres_War,genres_Western,company_portfolio,company_1492 Pictures,company_2929 Productions,company_40 Acres & A Mule Filmworks,company_Aardman Animations,company_Alcon Entertainment,company_Alliance Films,company_Amblin Entertainment,company_American Zoetrope,company_Anonymous Content,company_Artisan Entertainment,company_Atlas Entertainment,company_BBC Films,company_BIM Distribuzione,company_Bac Films,...,company_TF1 Films Production,company_The Javelina Film Company,company_The Mark Gordon Company,company_The Weinstein Company,company_The Zanuck Company,company_Thinkfilm,company_This Is That Productions,company_Tig Productions,company_Tim Burton Productions,company_Touchstone Pictures,company_Tree Line Films,company_TriStar Pictures,company_Twentieth Century Fox Film Corporation,company_UK Film Council,company_USA Films,company_United Artists,company_Universal Pictures,company_Universal Studios,company_Vertigo Entertainment,company_Village Roadshow Pictures,company_Virtual Studios,company_Walt Disney,company_Walt Disney Animation Studios,company_Walt Disney Pictures,company_Walt Disney Productions,company_Warner Bros.,company_Warner Bros. Family Entertainment,company_Warner Bros. Pictures,company_Weed Road Pictures,company_Wild Bunch,company_Wildwood Enterprises,company_WingNut Films,company_Wonderland Sound and Vision,company_Working Title Films,company_Worldview Entertainment,company_Yari Film Group,company_Zentropa Entertainments,original_language,profitable,foreign
0,237000000,7.2,162.0,16.0,2009,12,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False
1,300000000,6.9,169.0,82.0,2007,5,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,82.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False
2,245000000,6.3,148.0,144.0,2015,10,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True
3,250000000,7.6,165.0,14.0,2012,7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False
4,260000000,6.1,132.0,82.0,2012,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,27000,7.4,92.0,1.0,1994,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True
4788,12000,6.2,93.0,1.0,1972,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True
4792,20000,7.4,111.0,1.0,1997,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
4796,7000,6.9,77.0,1.0,2004,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True


In [323]:
#pca = PCA(n_components=20)
#X = pca.fit_transform(dfm[num_cols + cat_cols + cat_cols2])
X = dfm[num_cols + cat_cols2]
y = np.array(dfm[target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

print(X_train, X_test, y_train, y_test)

        budget  vote_average  runtime  ...  original_language  profitable  foreign
2920  11000000           7.3    106.0  ...                  1        True    False
4669    210000           6.7     80.0  ...                  1        True     True
2203  30000000           6.4    101.0  ...                  1       False     True
940   50000000           6.3    128.0  ...                  1       False     True
4219   1000000           5.9     97.0  ...                  1        True     True
...        ...           ...      ...  ...                ...         ...      ...
1178  40000000           6.2     90.0  ...                  1        True    False
1218  40000000           5.4     95.0  ...                  1       False     True
1418  35000000           6.3    155.0  ...                  1       False     True
913   70000000           6.4    104.0  ...                  1        True    False
4579    400000           7.8     91.0  ...                  1        True     True

[16

In [0]:
def print_metrics(y_preds, y):
    print(f'R^2: {r2_score(y_preds, y)}')
    print(f'MSE: {mean_squared_error(y_preds, y)}')
    #print(f'Accuracy: {accuracy_score()}')

In [325]:
# Standart linearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

print_metrics(lr.predict(X_test), y_test)

R^2: 0.37422447758963195
MSE: 1.3447395647794936e+16


In [326]:
#Ridge LinearRegression
rlr = Ridge(alpha=1)
rlr.fit(X_train, y_train)

print_metrics(rlr.predict(X_test), y_test)

R^2: 0.37342113252603815
MSE: 1.344737458633567e+16


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.53121e-17): result may not be accurate.
  overwrite_a=True).T


In [327]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

print_metrics(gbr.predict(X_test), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R^2: 0.542348319675182
MSE: 1.0853051010724258e+16


In [328]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

print_metrics(rfr.predict(X_test), y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R^2: 0.5512685137970726
MSE: 1.1079443563444794e+16


In [329]:
print(target)
cross_validate(Ridge(), X, dfm[target], cv=5, 
               scoring={'r2_score': make_scorer(r2_score, ), 
                        'mean_squared_error': make_scorer(mean_squared_error)})

['revenue']


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.54873e-17): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.50609e-17): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.3629e-17): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.19283e-17): result may not be accurate.
  overwrite_a=True).T


{'fit_time': array([0.00636792, 0.00748086, 0.0072403 , 0.00714183, 0.00708365]),
 'score_time': array([0.00408053, 0.00374436, 0.00366926, 0.00371075, 0.00373363]),
 'test_mean_squared_error': array([4.86326106e+16, 6.43528400e+15, 4.49229949e+15, 6.48238868e+15,
        6.52646850e+15]),
 'test_r2_score': array([ 0.4386604 ,  0.47455498,  0.35508593,  0.06133926, -2.12038599])}

In [330]:
cross_validate(RandomForestRegressor(), X, dfm[target], cv=5, 
               scoring={'r2_score': make_scorer(r2_score, ), 
                        'mean_squared_error': make_scorer(mean_squared_error)})

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

{'fit_time': array([1.03649879, 1.01943088, 0.9906826 , 0.99749398, 0.99549532]),
 'score_time': array([0.01540899, 0.01733851, 0.01888633, 0.01734471, 0.01491094]),
 'test_mean_squared_error': array([5.66060129e+16, 6.17452877e+15, 3.45304809e+15, 4.95151642e+15,
        5.84619829e+15]),
 'test_r2_score': array([ 0.34662778,  0.49584581,  0.50428076,  0.28301213, -1.79514032])}

In [0]:
from sklearn.model_selection import GridSearchCV

In [332]:
%%time
gbr_grid_search = GridSearchCV(GradientBoostingRegressor(), 
                               [{'n_estimators': [100, 150, 440], 'learning_rate': [0.01, 0.05, 0.1, 0.15]}],
                               cv=5,
                               error_score=make_scorer(mean_squared_error),
                               verbose=10)
gbr_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.592, total=   0.2s
[CV] learning_rate=0.01, n_estimators=100 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  learning_rate=0.01, n_estimators=100, score=0.599, total=   0.2s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.650, total=   0.2s
[CV] learning_rate=0.01, n_estimators=100 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  learning_rate=0.01, n_estimators=100, score=0.575, total=   0.2s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.545, total=   0.2s
[CV] learning_rate=0.01, n_estimators=150 ............................


...

[CV]  learning_rate=0.15, n_estimators=440, score=0.733, total=   0.7s
[CV] learning_rate=0.15, n_estimators=440 ............................


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  learning_rate=0.15, n_estimators=440, score=0.687, total=   0.7s


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   22.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 22.3 s, sys: 52 ms, total: 22.4 s
Wall time: 22.4 s


In [333]:
print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

{'learning_rate': 0.05, 'n_estimators': 150}
0.7264435457496294
GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=150,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


#Change task to Classification

In [337]:
target = ['profitable']

X = dfm[num_cols + cat_cols2]
y = np.array(dfm[target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

print(y_train, y_test)

[[ True]
 [ True]
 [False]
 ...
 [False]
 [ True]
 [ True]] [[False]
 [ True]
 [False]
 ...
 [False]
 [ True]
 [ True]]


In [347]:
rfr = RandomForestClassifier()
rfr.fit(X_train, y_train)

accuracy_score(np.array(y_test), rfr.predict(X_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.6439628482972136

In [349]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

accuracy_score(np.array(y_test), gbc.predict(X_test))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6526315789473685

In [350]:
knc = KNeighborsClassifier()
knc.fit(X_train, y_train)

accuracy_score(np.array(y_test), knc.predict(X_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


0.5436532507739938

In [352]:
lgr = LogisticRegression()
lgr.fit(X_train, y_train)

accuracy_score(np.array(y_test), lgr.predict(X_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.534984520123839